In [116]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
path =  configParser.get('stackOutputImages', 'path')

In [117]:
from glob import glob
model_dirs = glob(path + '*', recursive = True)
model_dirs

['/media/gamal/Passport/Evaluations/Audio Only',
 '/media/gamal/Passport/Evaluations/Full Input',
 '/media/gamal/Passport/Evaluations/No Audio',
 '/media/gamal/Passport/Evaluations/No Image Guide',
 '/media/gamal/Passport/Evaluations/No Speaker Embedding']

In [118]:
import pandas as pd
from pathlib import Path

df = pd.DataFrame(columns = ['model_name','cond_scale','id','path'])
i = 0
cond_scale_values = [0,2,4,6,8]
gen_faces_folder =  configParser.get('stackOutputImages', 'gen_faces_folder')
for model_dir in model_dirs:
    model_name = os.path.basename(os.path.normpath(model_dir))
    for cond_scale_value in cond_scale_values:
        dir = model_dir + '/' + str(cond_scale_value) + '/' + gen_faces_folder
        for root, dirs, files in os.walk(dir):

            for file in files:
                id = os.path.basename(Path(root).parent.absolute())
                df.loc[i] = [model_name,str(cond_scale_value),id,root + '/' + file]
                #print(model_name + ' ' + str(cond_scale_value) + ' ' + id + ' ' + root + '/' + file)
                i += 1


In [119]:
df.head()

,model_name,cond_scale,id,path
0,Audio Only,0,id00017,/media/gamal/Passport/Evaluations/Audio Only/0...
1,Audio Only,0,id00061,/media/gamal/Passport/Evaluations/Audio Only/0...
2,Audio Only,0,id00081,/media/gamal/Passport/Evaluations/Audio Only/0...
3,Audio Only,0,id00154,/media/gamal/Passport/Evaluations/Audio Only/0...
4,Audio Only,0,id00419,/media/gamal/Passport/Evaluations/Audio Only/0...


In [120]:
faces_folder =  configParser.get('stackOutputImages', 'faces_folder')
model_name = os.path.basename(os.path.normpath(faces_folder))
dir = faces_folder
for root, dirs, files in os.walk(dir):
    for file in files:
        id = os.path.basename(Path(root).parent.absolute())
        df.loc[i] = [model_name,None,id,root + '/' + file]
        #print(model_name + ' ' + str(cond_scale_value) + ' ' + id + ' ' + root + '/' + file)
        i += 1


In [121]:
df.tail()

,model_name,cond_scale,id,path
3064,Original Face,None,id09017,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3065,Original Face,None,id01509,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3066,Original Face,None,id02725,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3067,Original Face,None,id04232,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3068,Original Face,None,id05714,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...


In [122]:
image_guides_folder =  configParser.get('stackOutputImages', 'image_guides_folder')
model_name = os.path.basename(os.path.normpath(image_guides_folder))
dir = image_guides_folder
for root, dirs, files in os.walk(dir):
    for file in files:
        id = os.path.basename(Path(root).parent.absolute())
        df.loc[i] = [model_name,None,id,root + '/' + file]
        #print(model_name + ' ' + str(cond_scale_value) + ' ' + id + ' ' + root + '/' + file)
        i += 1

In [123]:
df.tail()

,model_name,cond_scale,id,path
3182,Image Guide,None,id03839,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
3183,Image Guide,None,id01509,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
3184,Image Guide,None,id03862,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
3185,Image Guide,None,id04950,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
3186,Image Guide,None,id07312,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...


In [124]:
ids_unique = df['id'].unique()
ids_unique
models = ['Original Face','Image Guide','Full Input', 'No Image Guide',
          'No Speaker Embedding', 'No Audio','Audio Only' ]

In [125]:
import sys
from PIL import Image
import matplotlib.pyplot as plt
import textwrap 
import matplotlib
matplotlib.use('Agg')
def stack_images(stack_these,results_folder):

  f, axarr = plt.subplots(1,len(stack_these),dpi=2400)
  plt.subplots_adjust(left=0,
                    bottom=0, 
                    right=2, 
                    top=1, 
                    wspace=0, 
                    hspace=1)

  for i in range(len(stack_these)):
    axarr[i].axis("off")
    axarr[i].set_title("\n".join(textwrap.wrap(stack_these[i][0], 15)) + 
                       '\n' + stack_these[i][1],
                       fontsize=3,loc='center', wrap=True,y=0.9)
    axarr[i].imshow(Image.open( stack_these[i][2])    )

  f.savefig(results_folder+'/'+stack_these[i][3]+'.png',bbox_inches = 'tight')
  plt.close('all')


In [126]:
def stack_images_hor_and_ver(stack_these,no_of_columns,results_file):
  no_of_rows = len(stack_these)//no_of_columns
  f, axarr = plt.subplots(no_of_rows,no_of_columns,dpi=2400)
  f.subplots_adjust(left=0,
                    bottom=0, 
                    right=1, 
                    top=1, 
                    wspace=0, 
                    hspace=0)
  i = 0
  j = 0
  for index, row in stack_these.iterrows():
    #print(str(i) + ' ' + str(j))
    axarr[j][i].axis("off")
    if(j==0):
      axarr[j][i].set_title("\n".join(textwrap.wrap(row['model_name'], 12)),
                        fontsize=8,loc='center', wrap=True)
    axarr[j][i].imshow(Image.open( row['path']))
    i = i + 1
    if(i >= no_of_columns):
      i = 0
      j = j + 1


  f.savefig(results_file,bbox_inches = 'tight')
  plt.close('all')

In [127]:
best_results_file =  configParser.get('stackOutputImages', 'best_results_file')
models = ['Original Face','Image Guide','Full Input', 'No Image Guide',
          'No Speaker Embedding', 'No Audio','Audio Only' ]
#Ethnicity conversion & age change
#ids_and_cond_scales = [
#         ['id06209',[None,None,'4','4','6','4','2']], # take image guide into consi
#        ['id02542',[None,None,'4','4','4','8','4']],    # audio overpowers image guide
#               ['id08456',[None,None,'2','2','8','8','4']], #ignore image guide and age wrong
#               ]

# Good
ids_and_cond_scales = [
                ['id01000',[None,None,'4','2','2','6','2']], 
                ['id02181',[None,None,'6','6','2','2','4']],   
               ['id02725',[None,None,'4','2','8','2','2' ]], 
               ['id04295',[None,None,'2','6','6','6','4']], 
               ['id05714',[None,None,'2','2','2','4','4']], 
               ['id08392',[None,None,'6','6','2','8','4']], 
               ['id08696',[None,None,'8','6','4','8','4']], 
               ]

df1 = df.iloc[:0].copy()
for id_and_cond_scales in ids_and_cond_scales:
    id_to_draw = id_and_cond_scales[0]
    cond_scales = id_and_cond_scales[1]
    i = 0
    for model in models:
        cond_scale = cond_scales[i]
        i = i + 1
        if(cond_scale == None):
            df_filtered = df.loc[(df['id'] == id_to_draw) 
                            & (df['model_name'] == model)
                            & (df['cond_scale'].isna())]
        else:
            df_filtered = df.loc[(df['id'] == id_to_draw) 
            & (df['model_name'] == model)
            & (df['cond_scale'] == cond_scale)]
        df1 = pd.concat([df1,df_filtered])


In [128]:
stack_images_hor_and_ver(df1,len(models),best_results_file)

In [129]:
df1

,model_name,cond_scale,id,path
2961,Original Face,None,id01000,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3079,Image Guide,None,id01000,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
836,Full Input,4,id01000,/media/gamal/Passport/Evaluations/Full Input/4...
1898,No Image Guide,2,id01000,/media/gamal/Passport/Evaluations/No Image Gui...
2488,No Speaker Embedding,2,id01000,/media/gamal/Passport/Evaluations/No Speaker E...
1544,No Audio,6,id01000,/media/gamal/Passport/Evaluations/No Audio/6/V...
128,Audio Only,2,id01000,/media/gamal/Passport/Evaluations/Audio Only/2...
2981,Original Face,None,id02181,/media/gamal/Passport/Datasets/VoxCeleb2Test/O...
3161,Image Guide,None,id02181,/media/gamal/Passport/Datasets/VoxCeleb2Test/I...
974,Full Input,6,id02181,/media/gamal/Passport/Evaluations/Full Input/6...



results_folder =  configParser.get('stackOutputImages', 'results_folder')
for id_unique in ids_unique:
    stack_these = []
    for model in models:
        df_filtered = df.loc[(df['id'] == id_unique) 
                        & (df['model_name'] == model)
                        & (df['cond_scale'].isna())]
        if(not df_filtered.empty):
            stack_these.append([model,'',df_filtered.iloc[0]['path'],id_unique])
        for cond_scale_value in cond_scale_values:
            df_filtered = df.loc[(df['id'] == id_unique) 
                                 & (df['model_name'] == model)
                                 & (df['cond_scale'] == str(cond_scale_value))]
            if(not df_filtered.empty):
                stack_these.append([model,str(cond_scale_value),
                                    df_filtered.iloc[0]['path'],id_unique])
    stack_images(stack_these,results_folder)